# <div align="center" style="color: #ff5733;">Complete Customer 360 Data</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)

# Demographic Data

## First Query

In [2]:
sq = """
WITH
  base AS (
  SELECT
    DISTINCT cust_id,
    kyc_status,
    DATE(created_dt) onboardingDate
    , date_add(date(created_dt), INTERVAL 120 day) day_120
    , date_add(date(created_dt), INTERVAL 150 day) day_150
  FROM
    `dl_customers_db_raw.tdbk_customer_mtb`
  WHERE
    1=1
    AND DATE(created_dt) >= '2023-01-01'
    AND DATE(created_dt) <= '2024-03-31'
    AND cust_id IS NOT NULL
    AND kyc_status != 'BKYC'),
  educate AS (
  SELECT
    DISTINCT edu.digitalLoanAccountId,
    edu.education_id,
    edu1.description
  FROM
    `prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_purpose` edu
  INNER JOIN (
    SELECT
      id,
      description
    FROM
      dl_loans_db_raw.tdbk_loan_lov_mtb
    WHERE
      module = 'Education') edu1
  ON
    edu.education_id = edu1.id ),
  educate2 AS (
  SELECT
    *,
    ROW_NUMBER() OVER(PARTITION BY digitalLoanAccountId ORDER BY education_id DESC) rnk
  FROM
    educate),
  educate3 AS (
  SELECT
    *
  FROM
    educate2
  WHERE
    rnk = 1),
  educationdf AS (
  SELECT
    DISTINCT lmt.customerId,
    educate3.education_id,
    educate3.description Education_Type,
    educate3.rnk educationrnk,
    ROW_NUMBER() OVER(PARTITION BY lmt.customerId ORDER BY education_id DESC) rnk
  FROM
    educate3
  INNER JOIN
    prj-prod-dataplatform.risk_credit_mis.loan_master_table lmt
  ON
    lmt.digitalLoanAccountId = educate3.digitalLoanAccountId
  WHERE
    lmt.customerId IN (
    SELECT
      CAST(cust_id AS numeric)
    FROM
      base) ),
  educationdf1 AS (
  SELECT
    customerId,
    education_id,
    Education_Type
  FROM
    educationdf
  WHERE
    rnk = 1 ),
  ac_created AS ( ## Base DATA OF onboarded users
  SELECT
    DISTINCT product,
    CASE
      WHEN product = 'LOAN1.0' THEN 'Borrow'
      WHEN product = 'TSA1.0' THEN 'Save'
      WHEN product = 'TSAE1.0' THEN 'Explore'
      ELSE product
  END
    type,
    cust_id,
    user_id,
    device_id,
    created_dt,
    gender,
    mobile_no,
    birthplace,
  FROM
    prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb
  WHERE
    1=1
    AND CAST(cust_id AS numeric) IN (
    SELECT
      CAST(cust_id AS numeric)
    FROM
      base) ),
  clientdemodf1 AS (
  SELECT
    ac.cust_id,
    ac.user_id,
    ac.device_id,
    ac.created_dt,
    ac.gender,
    ac.birthplace,
    ac.mobile_no,
    ac.product,
    ac.type,
    t4.GeneralData.telephony_info__network_operator_name telcoProvider
  FROM
    ac_created ac
  INNER JOIN
    prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_customer_details tlcd
  ON
    tlcd.custId = ac.cust_id
  INNER JOIN
    `prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3
  ON
    CAST(tlcd.credolabRefNumber AS string) = CAST(t3.refno AS string)
  LEFT JOIN
    `prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4
  ON
    t3.refno = t4.deviceId
  WHERE
    CAST(ac.cust_id AS numeric) IN (
    SELECT
      CAST(cust_id AS numeric)
    FROM
      base) ),
  cddf AS (
  SELECT
    custid,
    dateOfBirth,
    signUpAccNo,
    kycStatus,
    addressline1,
    city,
    province,
    barangay,
    provinceCode,
    postalcode,
    mailingCity,
    mailingPostalCode,
    mailingProvince,
    natureofwork,
    employmentstatus,
    mobileOs,
    docType,
    onboardingDate,
    created_dt,
    ABS(TIMESTAMP_DIFF( PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%E6S', onboardingDate), CAST(created_dt AS TIMESTAMP), MINUTE )) AS Minutestakentoonboard,
    companyName,
    industry,
    device_dtl,
  FROM
    `prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_customer_details`
  WHERE
    CAST(custId AS NUMERIC) IN (
    SELECT
      CAST(cust_id AS numeric)
    FROM
      base) ),
  regiondf AS (
  SELECT
    DISTINCT a.user_id,
    CAST(a.cust_Id AS numeric) cust_id,
    hm_postalcode,
    hm_barangay,
    hm_city,
    hm_province,
    CASE
      WHEN hm_province = 'KALINGA' THEN 'CAR – Cordillera Administrative Region'
      WHEN hm_province = 'OCCIDENTAL MINDORO' THEN 'MIMAROPA Region'
      WHEN hm_province = 'ORIENTAL MINDORO' THEN 'MIMAROPA Region'
      WHEN hm_province = 'ORODNIM' THEN 'MIMAROPA Region'
      WHEN hm_province = 'SAMAR (WESTERN SAMAR)' THEN 'Region VIII – Eastern Visayas'
      WHEN hm_province = 'ZAMBOANGA SIBUGAY' THEN 'Region IX – Zamboanga Peninsula'
      ELSE Region_name
  END
    Region_Name
  FROM
    prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb a
  LEFT JOIN
    prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_add_mtb f
  ON
    a.user_id = f.user_id
  LEFT JOIN
    prj-prod-dataplatform.dap_ds_poweruser_playground.region_mappings h
  ON
    LOWER(f.hm_province) = LOWER(h.province)
  WHERE
    CAST(a.cust_Id AS numeric) IN (
    SELECT
      CAST(cust_id AS numeric)
    FROM
      base) ),
  locationdf AS (
  SELECT
    customer_id,
    CASE
      WHEN event_description = 'Onboarding' THEN latitude
  END
    AS Onboarding_latitude,
    CASE
      WHEN event_description = 'Onboarding' THEN longitude
  END
    AS Onboarding_longitude,
    CASE
      WHEN event_description = 'Loan Journey' THEN latitude
  END
    AS loanjourney_latitude,
    CASE
      WHEN event_description = 'Loan Journey' THEN longitude
  END
    AS loanjouney_longitude,
    CASE
      WHEN event_description = 'Apigee Logs' THEN latitude
  END
    AS ApigeeLogs_latitude,
    CASE
      WHEN event_description = 'Apigee Logs' THEN longitude
  END
    AS ApigeeLogs_longitude
  FROM
    `prj-prod-dataplatform.risk_mart.customer_gps_location`
  WHERE
    CAST(customer_id AS numeric) IN (
    SELECT
      CAST(cust_id AS numeric)
    FROM
      base)
  QUALIFY
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY gps_collection_date DESC) = 1 ),
  loandf AS (
  SELECT
    customerId,
    maritalStatus,
    dependentsCount,
    monthlyIncome,
    startApplyDateTime,
  FROM
    prj-prod-dataplatform.risk_credit_mis.loan_master_table
  WHERE
    (maritalStatus IS NOT NULL
      OR dependentsCount IS NOT NULL
      OR monthlyIncome IS NOT NULL)
    AND customerId IN (
    SELECT
      CAST(cust_id AS numeric)
    FROM
      base)
  QUALIFY
    ROW_NUMBER() OVER (PARTITION BY customerId ORDER BY startApplyDateTime DESC) = 1 ),
  part1df AS (
  SELECT
    base.cust_id,
    base.onboardingDate,
    base.day_120,
    base.kyc_status,
    CASE
      WHEN educationdf1.customerId IS NULL THEN 1
      ELSE 0
  END
    educationdataflag,
    educationdf1.education_id,
    educationdf1.Education_Type,
    CASE
      WHEN clientdemodf1.cust_id IS NULL THEN 1
      ELSE 0
  END
    clientdemographicflag,
    clientdemodf1.user_id,
    clientdemodf1.device_id,
    clientdemodf1.created_dt,
    clientdemodf1.gender,
    clientdemodf1.birthplace,
    clientdemodf1.mobile_no,
    clientdemodf1.product,
    clientdemodf1.type,
    clientdemodf1.telcoProvider,
    CASE
      WHEN cddf.custid IS NULL THEN 1
      ELSE 0
  END
    cdd_flag,
    cddf.dateOfBirth,
    cddf.signUpAccNo,
    cddf.kycStatus kycStatusfromCDD,
    cddf.addressline1,
    cddf.city,
    cddf.province,
    cddf.barangay,
    cddf.provinceCode,
    cddf.postalcode,
    cddf.mailingCity,
    cddf.mailingPostalCode,
    cddf.mailingProvince,
    cddf.natureofwork,
    cddf.employmentstatus,
    cddf.mobileOs,
    cddf.docType,
    cddf.Minutestakentoonboard,
    cddf.companyName,
    cddf.industry,
    cddf.device_dtl,
    regiondf.hm_postalcode,
    regiondf.hm_barangay,
    regiondf.hm_city,
    regiondf.hm_province,
    regiondf.Region_name,
    CASE
      WHEN locationdf.customer_id IS NULL THEN 1
      ELSE 0
  END
    locationdfflag,
    locationdf.Onboarding_latitude,
    locationdf.Onboarding_longitude,
    locationdf.loanjourney_latitude,
    locationdf.loanjouney_longitude,
    locationdf.ApigeeLogs_latitude,
    locationdf.ApigeeLogs_longitude,
    CASE
      WHEN loandf.customerId IS NULL THEN 1
      ELSE 0
  END
    loandfFlag,
    loandf.maritalStatus,
    loandf.dependentsCount,
    loandf.monthlyIncome,
    loandf.startApplyDateTime
  FROM
    base
  LEFT JOIN
    educationdf1
  ON
    CAST(base.cust_id AS numeric) = educationdf1.customerId
  LEFT JOIN
    clientdemodf1
  ON
    base.cust_id = clientdemodf1.cust_id
  LEFT JOIN
    cddf
  ON
    base.cust_id = cddf.custid
  LEFT JOIN
    regiondf
  ON
    CAST(base.cust_id AS numeric) = regiondf.cust_id
  LEFT JOIN
    locationdf
  ON
    base.cust_id = locationdf.customer_id
  LEFT JOIN
    loandf
  ON
    CAST(base.cust_id AS numeric) = loandf.customerId )
SELECT
  *
FROM
  part1df;
"""

part1df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 7200fe7f-931c-4e72-ab45-662e4492c2ac successfully executed: |          |
Downloading: 100%|██████████|


In [3]:
print(f"The shape of the part1df is:\t {part1df.shape}")

The shape of the part1df is:	 (601874, 55)


## Part2 query

In [4]:
sq = """  
WITH 
  base AS (
  SELECT
    DISTINCT cust_id,
    kyc_status,
    DATE(created_dt) onboardingDate
  FROM
    `dl_customers_db_raw.tdbk_customer_mtb`
  WHERE
    1=1
    AND DATE(created_dt) >= '2023-01-01'
    AND DATE(created_dt) <= '2024-03-31'
    AND cust_id IS NOT NULL
    AND kyc_status != 'BKYC'),
ac_created AS 
    (
    ## Base data of onboarded users
    SELECT DISTINCT 
    product,
    CASE 
        WHEN product = 'LOAN1.0' THEN 'Borrow'
        WHEN product = 'TSA1.0' THEN 'Save'
        WHEN product = 'TSAE1.0' THEN 'Explore'
        ELSE product END type,
    cust_id,
    user_id,
    device_id,
    created_dt,
    gender,
    mobile_no,
    FROM prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb
    WHERE 1=1
    AND cast(cust_id as numeric) in (select cast(cust_id as numeric) from base)
    AND DATE(created_dt) >= '2023-01-01'
    AND DATE(created_dt) >= '2024-03-31'
    ORDER BY 4
    )

    , f_loan AS 
    (
    ## First applied loan of a customer 
    SELECT DISTINCT customerId, new_loan_type, applicationStatus, disbursementdatetime, startApplydatetime
    FROM `risk_credit_mis.loan_master_table`
    WHERE 1=1
    AND customerId in (select cast(cust_id as numeric) from base)
    AND startApplyDatetime >= '2023-01-01'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY customerId ORDER BY startApplyDatetime ASC) =1
    )


    , deposit_balance AS 
    (
    ## Gets the balance of a customer on their 90th day upon account opening
        ## For the 1st condition of the balance of a customer upon the observation date
    SELECT DISTINCT a.ofdateopened,a.ofcustomerid, a.ofstandardaccountid,account_type,balanceDateAsOf, clearedbalance,
    FROM `prj-prod-dataplatform.core_raw.customer_accounts` a
    JOIN `risk_mart.customer_balance` b ON a.ofcustomerid = b.client_id AND a.ofstandardaccountid = b.accountid
    WHERE 1=1
    AND cast(a.ofcustomerid as numeric) in (select cast(cust_id as numeric) from base)
    -- AND clearedbalance >= 100
    -- AND ofcustomerid IN ('1514439','1252865','1248952')
    QUALIFY ROW_NUMBER() OVER (PARTITION BY ofcustomerid, ofstandardaccountid ORDER BY balancedateasof ASC) = 90
    )

    , af_link AS
    (
    ## To get the AF ID and Customer ID Link (using the first install of a customer)
    SELECT DISTINCT customer_user_id, appsflyer_id, media_source, partner, campaign
    FROM `appsflyer_raw.organic_in_app_events_report`
    WHERE 1=1
    and customer_user_id in (select cust_id from base)
    AND customer_user_id IS NOT NULL
    QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_user_id ORDER BY install_time ASC) = 1
    
    UNION ALL
    
    SELECT DISTINCT customer_user_id, appsflyer_id, media_source, partner, campaign
    FROM `appsflyer_raw.in_app_events_report`
    WHERE 1=1
    and customer_user_id in (select cust_id from base)
    AND customer_user_id IS NOT NULL
    QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_user_id ORDER BY install_time ASC) = 1
    )


    , demog_details AS 
    (
    ## Demographic Details
    SELECT DISTINCT
    a.user_id,
    a.cust_Id,
    a.device_id,
    b.id,
    c.description source_of_funds,
    d.description employment_status,
    monthly_income,
    hm_postalcode,
    hm_barangay,
    hm_city,
    hm_province,
    CASE
        WHEN hm_province = 'KALINGA' THEN 'CAR – Cordillera Administrative Region'
        WHEN hm_province = 'OCCIDENTAL MINDORO' THEN 'MIMAROPA Region'
        WHEN hm_province = 'ORIENTAL MINDORO' THEN  'MIMAROPA Region'
        WHEN hm_province = 'ORODNIM' THEN 'MIMAROPA Region'
        WHEN hm_province = 'SAMAR (WESTERN SAMAR)' THEN 'Region VIII – Eastern Visayas'
        WHEN hm_province = 'ZAMBOANGA SIBUGAY' THEN 'Region IX – Zamboanga Peninsula'
        ELSE Region_name END Region_Name
    FROM prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb a
    LEFT JOIN prj-prod-dataplatform.dl_dynamo_db_raw.tdbk_regfinancial_profile1 b ON a.device_id = b.deviceId
    LEFT JOIN prj-prod-dataplatform.dl_dynamo_db_raw.tdbk_source_of_funds_mtb c ON b.sourceOfFundKey = c.id  
    LEFT JOIN prj-prod-dataplatform.dl_dynamo_db_raw.tdbk_employment_status_mtb d ON b.employmentStatusKey = d.id
    LEFT JOIN prj-prod-dataplatform.dl_customers_db_raw.tdbk_cust_profile_mtb e ON a.user_id = e.user_id
    LEFT JOIN prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_add_mtb f ON a.user_id = f.user_id
    LEFT JOIN prj-prod-dataplatform.dap_ds_poweruser_playground.region_mappings h ON LOWER(f.hm_province) = LOWER(h.province)
    where cast(a.cust_Id as numeric) in (select cast(cust_id as numeric) from base)
    ),
  bb 
  as
    ## Base query used to connect to other features 
  (  SELECT DISTINCT
    base.onboardingDate registration_date,
    -- startApplyDateTime,
    -- ofdateopened,
    -- balancedateasof,
    base.cust_id,
    a.user_id,
    a.product,
    a.type,
    -- account_type,
    -- ofstandardaccountid,
    -- clearedbalance, 
    a.gender,
    customer_age,
    Region_Name,
    hm_province,
    hm_city,
    hm_barangay,
    hm_postalcode,
    source_of_funds,
    employment_status,
    CAST(monthly_income AS FLOAT64)*12.5 self_declared_annual_income,
    -- add the salary scale annual income
    -- double check if i can switch networks given the same number

    mobile_no,
    IFNULL(h.network_group,g.network_group) network_group,
    CASE
        WHEN g.network_group = 'Globe' AND h.network IS NULL THEN 'Prepaid'
        WHEN g.network_group = 'Globe' AND h.network IS NOT NULL THEN 'Postpaid'
        WHEN g.network_group = 'Smart' AND (LEFT(mobile_no,5) = '63920' OR LEFT(mobile_no,5) = '63918') THEN 'Postpaid'
        WHEN g.network_group NOT IN ('Globe','Smart') THEN NULL
        ELSE 'Prepaid' END network,
    CASE 
    ### social media ###
    -- anything from social
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%social%' THEN 'Social'
    ### direct ###
    -- fwb / referral
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'af_app_invites' THEN 'FWB'
    WHEN (Media_Source = 'invalid_media_source_name' OR Media_Source IS NULL) AND LOWER(Campaign) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%refer%' THEN 'FWB'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%moengage%' THEN 'FWB'
    -- apple search
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%apple search ads%' THEN 'AppleSearch'
    -- facebook / meta
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Facebook Ads%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%meta%' THEN 'Ads_Meta'
    WHEN COALESCE(Media_Source,Partner,Campaign) = 'facebook' THEN 'Ads_Meta'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) = 'restricted' THEN 'Ads_Meta'
    -- google
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%google%' THEN 'Ads_Google'
        -- tiktok
    WHEN COALESCE(Media_Source,Partner,Campaign) like '%bytedanceglobal%' THEN 'TikTok_Paid'
    ### affiliates ###
    -- pokkt source
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%pokkt%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%Tonik_CPA%' THEN 'Aff_Pokkt'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%campaign_name%' THEN 'Aff_Pokkt'
    -- tyr ads
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) IN ('tyrads_int','tjzymob_int','tyrads','ta_tonik_aos_ph') THEN 'Aff_TyrAds'
    -- sales doubler
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%salesdoubler%' THEN 'Aff_SalesDoubler'
    
    -- imoney
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%imoney%' THEN 'Aff_iMoney'
    -- moneymax
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%moneymax%' THEN 'Aff_MoneyMax'
    -- jeff
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%jeff%' THEN 'Aff_Jeff'
    -- shareit
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%shareit%' THEN 'Aff_ShareIt'
    
    -- appnext
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%appnext%' THEN 'Aff_AppNext'
    -- mediadonuts / entravision
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%mediadonuts%' THEN 'Aff_Mediadonuts'
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%entravision%' THEN 'Aff_Mediadonuts'
    -- shopback
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%shopback_int%' THEN 'Aff_Shopback'
    -- deepsea
    WHEN LOWER(COALESCE(Media_Source,Partner,Campaign)) LIKE '%deepsea%' THEN 'Aff_DeepSea'
    -- avow
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%avow%' THEN 'Aff_AvowTech'
    -- avow
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%vivo%' THEN 'Aff_Vivo'
    -- 711
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%SevenEleven%' THEN 'SevenEleven'
    ## organic ##
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('af_banner') THEN 'af_Banner'
    WHEN COALESCE(Media_Source,Partner,Campaign) LIKE '%sendgrid%' THEN 'Organic'
    WHEN COALESCE(Media_Source,Partner,Campaign) IN ('Website') THEN 'Website'
    
    ## owned media ##
    WHEN media_source IN ('PRFlexLoans_WebAndroidPressRelease') THEN 'Owned Media'
    
    WHEN media_source IN ('invalid_media_source_name') THEN 'Invalid Media Source'
    WHEN media_source IN ('None') THEN 'Broken OneLink'
    WHEN media_source IS NULL AND partner IS NULL AND campaign IS NULL THEN 'Organic'
    ELSE COALESCE(media_source,partner,campaign) END as Source,
    media_source,
    partner,
    campaign,
    
    FROM base 
    LEFT JOIN ac_created a on a.cust_id = base.cust_id
    LEFT JOIN f_loan b ON a.cust_id  = CAST(b.customerId AS STRING)
    LEFT JOIN deposit_balance c ON a.cust_id = c.ofcustomerid
    LEFT JOIN af_link d ON cast(a.cust_id as string) = d.customer_user_id
    LEFT JOIN prj-prod-dataplatform.dl_customers_db_derived.Tdbk_customer_mtb_age_derived e ON a.cust_id = e.cust_id
    LEFT JOIN demog_details f ON a.cust_id = f.cust_id
    LEFT JOIN `prj-prod-dataplatform.manual_source_extracts.mobile_carrier_mapping` g
    ON LEFT(RIGHT(mobile_no,LENGTH(mobile_no)-2),3) = CAST(g.number_prefix AS STRING)
    LEFT JOIN `prj-prod-dataplatform.manual_source_extracts.mobile_carrier_mapping` h 
    ON LEFT(RIGHT(mobile_no,LENGTH(mobile_no)-2),4) = CAST(h.number_prefix AS STRING)
    WHERE 1=1
  ),
  part2df 
  as
  (select cust_id, network_group, Source,	media_source,	partner,campaign 
  , row_number() over(partition by cust_id order by registration_date) rnk
  from bb)
  select * from part2df where rnk = 1 ;
  """
  
part2df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 9ae5e612-431f-48de-b3b2-a4001f7176b1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [5]:
print(f"The shape of part2df is:\t {part2df.shape}")

The shape of part2df is:	 (600939, 7)


In [6]:
part1df.columns

Index(['cust_id', 'onboardingDate', 'day_120', 'kyc_status',
       'educationdataflag', 'education_id', 'Education_Type',
       'clientdemographicflag', 'user_id', 'device_id', 'created_dt', 'gender',
       'birthplace', 'mobile_no', 'product', 'type', 'telcoProvider',
       'cdd_flag', 'dateOfBirth', 'signUpAccNo', 'kycStatusfromCDD',
       'addressline1', 'city', 'province', 'barangay', 'provinceCode',
       'postalcode', 'mailingCity', 'mailingPostalCode', 'mailingProvince',
       'natureofwork', 'employmentstatus', 'mobileOs', 'docType',
       'Minutestakentoonboard', 'companyName', 'industry', 'device_dtl',
       'hm_postalcode', 'hm_barangay', 'hm_city', 'hm_province', 'Region_name',
       'locationdfflag', 'Onboarding_latitude', 'Onboarding_longitude',
       'loanjourney_latitude', 'loanjouney_longitude', 'ApigeeLogs_latitude',
       'ApigeeLogs_longitude', 'loandfFlag', 'maritalStatus',
       'dependentsCount', 'monthlyIncome', 'startApplyDateTime'],
      dtype='o

In [7]:
part2df.columns

Index(['cust_id', 'network_group', 'Source', 'media_source', 'partner',
       'campaign', 'rnk'],
      dtype='object')

In [8]:
maindf = part1df.merge(part2df, on = 'cust_id', how = 'left')
print(f"The shape of maindf is:\t{maindf.shape}")

The shape of maindf is:	(601874, 61)


In [9]:
maindf.columns

Index(['cust_id', 'onboardingDate', 'day_120', 'kyc_status',
       'educationdataflag', 'education_id', 'Education_Type',
       'clientdemographicflag', 'user_id', 'device_id', 'created_dt', 'gender',
       'birthplace', 'mobile_no', 'product', 'type', 'telcoProvider',
       'cdd_flag', 'dateOfBirth', 'signUpAccNo', 'kycStatusfromCDD',
       'addressline1', 'city', 'province', 'barangay', 'provinceCode',
       'postalcode', 'mailingCity', 'mailingPostalCode', 'mailingProvince',
       'natureofwork', 'employmentstatus', 'mobileOs', 'docType',
       'Minutestakentoonboard', 'companyName', 'industry', 'device_dtl',
       'hm_postalcode', 'hm_barangay', 'hm_city', 'hm_province', 'Region_name',
       'locationdfflag', 'Onboarding_latitude', 'Onboarding_longitude',
       'loanjourney_latitude', 'loanjouney_longitude', 'ApigeeLogs_latitude',
       'ApigeeLogs_longitude', 'loandfFlag', 'maritalStatus',
       'dependentsCount', 'monthlyIncome', 'startApplyDateTime',
       'network

### Check the addresses and if blank get the value from other column. Drop the extra columns

In [10]:
columns_to_check = [
    ('city', 'hm_city'),
    ('province', 'hm_province'),
    ('barangay', 'hm_barangay'),
    ('postalcode', 'hm_postalcode')
]

# Loop through the columns and replace null values
# print("Loop through the columns and replace null values")
for col, hm_col in columns_to_check:
    maindf[col] = maindf[col].fillna(maindf[hm_col])

maindf.drop(columns = ['hm_postalcode', 'hm_barangay', 'hm_city',
    'hm_province'], inplace = True)

### Calculate the age

In [12]:
from datetime import datetime

def calculate_age_in_days(date_of_birth, dd=None):
    """
    Calculate age in days from the date of birth to the current date or a given date.
    
    Parameters:
    - date_of_birth (str): Date of birth in the format "dd MMM yyyy".
    - dd (datetime or None): A specific date to calculate age up to. If None, use the current date.
    
    Returns:
    - int or None: Age in days, or None if the input is invalid.
    """
    # Check if the date_of_birth is None or not a string
    if not isinstance(date_of_birth, str):
        return None  # Return None if the input is not a valid string

    try:
        # Parse the date of birth into a datetime object
        dob = datetime.strptime(date_of_birth, "%d %b %Y")
    except ValueError:
        return None  # Return None if the date string is invalid

    # Convert dd to datetime if it is provided and not already in datetime format
    if dd is not None:
        if isinstance(dd, datetime):
            dd = dd
        else:
            dd = datetime.combine(dd, datetime.min.time())  # Convert to datetime object
    else:
        dd = datetime.now()

    # Calculate the difference in days
    age_days = (dd - dob).days

    return age_days


def add_age_column(df, date_column='dateOfBirth', dd_column=None, new_column='AgeInDays'):
    """
    Add an age column to the DataFrame based on the date of birth and an optional date column.
    
    Parameters:
    - df (DataFrame): Input DataFrame.
    - date_column (str): Name of the column with date of birth.
    - dd_column (str or None): Name of the column with specific dates. If None, use the current date.
    - new_column (str): Name of the new column to be added for age in days.
    
    Returns:
    - DataFrame: DataFrame with the new age column.
    """
    # Apply the calculate_age_in_days function to the date column with the given or default date
    if dd_column:
        # Ensure the dd_column is converted to datetime
        df[dd_column] = pd.to_datetime(df[dd_column], errors='coerce').dt.to_pydatetime()
        df[new_column] = df.apply(lambda x: calculate_age_in_days(x[date_column], x[dd_column]), axis=1)
    else:
        df[new_column] = df[date_column].apply(calculate_age_in_days)
    
    return df


# Example usage:
# If 'dd_column' is specified, age will be calculated based on that date; otherwise, the current date is used
maindf = add_age_column(maindf, date_column='dateOfBirth', dd_column='day_120', new_column='AgeInDays')

# Convert age in days to age in years
maindf['Age'] = maindf['AgeInDays'] // 365


C:\Users\DwaipayanChakroborti\AppData\Local\Temp\ipykernel_18508\1739412040.py:55: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  df[dd_column] = pd.to_datetime(df[dd_column], errors='coerce').dt.to_pydatetime()


In [19]:
# def calculate_age_in_days(date_of_birth, dd):
#     from datetime import datetime
    
#     # Check if the date_of_birth is None or not a string
#     if not isinstance(date_of_birth, str):
#         return None  # Return None if the input is not a valid string

#     try:
#         # Parse the date of birth
#         dob = datetime.strptime(date_of_birth, "%d %b %Y")
#     except ValueError:
#         return None  # Return None if the date string is invalid

#     # Get the current date
#     current_date = datetime.now()
#     dd

#     # Calculate the difference in days
#     if len(dd) == 0:
#         age_days = (current_date - dob).days
#     else:
#         age_days = (dd - dob).days

#     return age_days


# def add_age_column(df, date_column='dateOfBirth', new_column='AgeInDays'):
#     # Apply the calculate_age_in_days function to the date column
#     df[new_column] = df[date_column].apply(calculate_age_in_days)
#     return df

# maindf['Age'] = maindf['dateOfBirth'].apply(calculate_age_in_days)
# maindf['Age'] = maindf['Age'] //365

### Creating column sourcGroups

In [13]:
source_groups_mapping = {
    'Ads_Google': 'Direct',
    'Ads_Meta': 'Direct',
    'Aff_AppNext': 'Affiliates',
    'Aff_AvowTech': 'Affiliates',
    'Aff_DeepSea': 'Affiliates',
    'Aff_Jeff': 'Affiliates',
    'Aff_Mediadonuts': 'Affiliates',
    'Aff_MoneyMax': 'Affiliates',
    'Aff_Pokkt': 'Affiliates',
    'Aff_SalesDoubler': 'Affiliates',
    'Aff_ShareIt': 'Affiliates',
    'Aff_Shopback': 'Affiliates',
    'Aff_TyrAds': 'Affiliates',
    'Aff_Vivo': 'Affiliates',
    'Aff_iMoney': 'Affiliates',
    'acesaatchi': 'Affiliates',
    'AppleSearch': 'Direct',
    'Broken OneLink': 'Others',
    'Email': 'Others',
    'FWB': 'Direct',
    'OpsEngine_post': 'Others',
    'Organic': 'Organic',
    'Owned Media': 'Others',
    'Partnership': 'Others',
    'SevenEleven': 'Others',
    'Social': 'Organic',
    'Test': 'Others',
    'TikTok_Paid': 'Direct',
    'Website': 'Organic',
    'af_Banner': 'Organic',
    'Tonik_PHL_default': 'Others'
}

# Assuming demogdf2 is your DataFrame
# Create the SourceGroups column by mapping the Source column to the corresponding groups
maindf['SourceGroups'] = maindf['Source'].map(source_groups_mapping)

In [14]:
# Step 1: Convert columns to datetime datatype
maindf['onboardingDate'] = pd.to_datetime(maindf['onboardingDate'])
maindf['startApplyDateTime'] = pd.to_datetime(maindf['startApplyDateTime'])

# Step 2: Find the earliest date between onboardingDate and startApplyDateTime
maindf['earliest_date'] = maindf[['onboardingDate', 'startApplyDateTime']].min(axis=1)

# Step 3: Calculate the difference in months from the earliest date to current date
current_date = pd.Timestamp.now()
maindf['months_difference'] = (current_date - maindf['earliest_date']).dt.days / 30.44  # Average days in a month

# Round the result to 2 decimal places
maindf['months_difference'] = maindf['months_difference'].round(2)

In [17]:
maindf.columns

Index(['cust_id', 'onboardingDate', 'day_120', 'kyc_status',
       'educationdataflag', 'education_id', 'Education_Type',
       'clientdemographicflag', 'user_id', 'device_id', 'created_dt', 'gender',
       'birthplace', 'mobile_no', 'product', 'type', 'telcoProvider',
       'cdd_flag', 'dateOfBirth', 'signUpAccNo', 'kycStatusfromCDD',
       'addressline1', 'city', 'province', 'barangay', 'provinceCode',
       'postalcode', 'mailingCity', 'mailingPostalCode', 'mailingProvince',
       'natureofwork', 'employmentstatus', 'mobileOs', 'docType',
       'Minutestakentoonboard', 'companyName', 'industry', 'device_dtl',
       'Region_name', 'locationdfflag', 'Onboarding_latitude',
       'Onboarding_longitude', 'loanjourney_latitude', 'loanjouney_longitude',
       'ApigeeLogs_latitude', 'ApigeeLogs_longitude', 'loandfFlag',
       'maritalStatus', 'dependentsCount', 'monthlyIncome',
       'startApplyDateTime', 'network_group', 'Source', 'media_source',
       'partner', 'campaign', 

In [16]:
maindf.drop(columns = 'AgeInDays', inplace = True)

### Rename the columns

In [18]:
# Read the Excel file (replace 'your_file.xlsx' with the actual file name)
excel_data = pd.read_excel(r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Customer_360_Data_Prep\SupportingFiles\Demographic_data_column_mapping.xlsx', sheet_name = 'Sheet1')
excel_data.columns
# Create a dictionary to map current column names to updated column names
rename_dict = dict(zip(excel_data['Column'], excel_data['Updated_Name']))

rename_dict

{'cust_id': 'cust_id',
 'onboardingDate': 'de_onboardingDate',
 'kyc_status': 'de_kyc_status',
 'educationdataflag': 'de_educationdataflag',
 'education_id': 'de_education_id',
 'Education_Type': 'de_Education_Type',
 'clientdemographicflag': 'de_clientdemographicflag',
 'user_id': 'de_user_id',
 'device_id': 'de_device_id',
 'created_dt': 'de_created_dt',
 'gender': 'de_gender',
 'birthplace': 'de_birthplace',
 'mobile_no': 'de_mobile_no',
 'product': 'de_product',
 'type': 'de_type',
 'telcoProvider': 'de_telcoProvider',
 'cdd_flag': 'de_cdd_flag',
 'dateOfBirth': 'de_dateOfBirth',
 'signUpAccNo': 'de_signUpAccNo',
 'kycStatusfromCDD': 'de_kycStatusfromCDD',
 'addressline1': 'de_addressline1',
 'city': 'de_city',
 'province': 'de_province',
 'barangay': 'de_barangay',
 'provinceCode': 'de_provinceCode',
 'postalcode': 'de_postalcode',
 'mailingCity': 'de_mailingCity',
 'mailingPostalCode': 'de_mailingPostalCode',
 'mailingProvince': 'de_mailingProvince',
 'natureofwork': 'de_natureof

#### Rename the column as per A_Demographic_data_bnb_update Excel Sheet

In [19]:
res = maindf.copy()

In [20]:
# Rename columns in the DataFrame 'res'
res.rename(columns=rename_dict, inplace=True)
res.columns

Index(['cust_id', 'de_onboardingDate', 'day_120', 'de_kyc_status',
       'de_educationdataflag', 'de_education_id', 'de_Education_Type',
       'de_clientdemographicflag', 'de_user_id', 'de_device_id',
       'de_created_dt', 'de_gender', 'de_birthplace', 'de_mobile_no',
       'de_product', 'de_type', 'de_telcoProvider', 'de_cdd_flag',
       'de_dateOfBirth', 'de_signUpAccNo', 'de_kycStatusfromCDD',
       'de_addressline1', 'de_city', 'de_province', 'de_barangay',
       'de_provinceCode', 'de_postalcode', 'de_mailingCity',
       'de_mailingPostalCode', 'de_mailingProvince', 'de_natureofwork',
       'de_employmentstatus', 'de_mobileOs', 'de_docType',
       'de_Minutestakentoonboard', 'de_companyName', 'de_industry',
       'de_device_dtl', 'de_Region_name', 'de_locationdfflag',
       'de_Onboarding_latitude', 'de_Onboarding_longitude',
       'de_loanjourney_latitude', 'de_loanjouney_longitude',
       'de_ApigeeLogs_latitude', 'de_ApigeeLogs_longitude', 'de_loandfFlag',
      

### Reshape the columns in dataframe

In [21]:
res = res[['cust_id', 'de_onboardingDate', 'de_kyc_status', 'de_educationdataflag',
       'de_education_id', 'de_Education_Type', 'de_clientdemographicflag',
       'de_user_id', 'de_device_id', 'de_created_dt', 'de_gender',
       'de_birthplace', 'de_mobile_no', 'de_product', 'de_type',
       'de_telcoProvider', 'de_cdd_flag', 'de_dateOfBirth', 'de_signUpAccNo',
       'de_kycStatusfromCDD', 'de_addressline1', 'de_city', 'de_province',
       'de_barangay', 'de_provinceCode', 'de_postalcode', 'de_mailingCity',
       'de_mailingPostalCode', 'de_mailingProvince', 'de_natureofwork',
       'de_employmentstatus', 'de_mobileOs', 'de_docType',
       'de_Minutestakentoonboard', 'de_companyName', 'de_industry',
       'de_device_dtl', 'de_Region_name', 'de_locationdfflag',
       'de_Onboarding_latitude', 'de_Onboarding_longitude',
       'de_loanjourney_latitude', 'de_loanjouney_longitude',
       'de_ApigeeLogs_latitude', 'de_ApigeeLogs_longitude', 'de_loandfFlag',
       'de_maritalStatus', 'de_dependentsCount', 'de_monthlyIncome',
       'de_startApplyDateTime', 'de_network_group', 'de_Source',
       'de_media_source', 'de_partner', 'de_campaign', 'de_rnk', 'de_Age',
       'de_SourceGroups', 'de_earliest_date', 'de_months_difference']].copy()

In [22]:
res.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Customer_360_Data_Prep\Data\FinalData\Demographic_Daily_Snapshot_Data_renamedcolumn_20240913.csv", index = False)

In [23]:
res = res.drop_duplicates(keep='first')
res.shape

(600939, 60)

In [24]:
res.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600939 entries, 0 to 601873
Data columns (total 60 columns):
 #   Column                    Non-Null Count   Dtype              
---  ------                    --------------   -----              
 0   cust_id                   600939 non-null  object             
 1   de_onboardingDate         600939 non-null  datetime64[ns]     
 2   de_kyc_status             600939 non-null  object             
 3   de_educationdataflag      600939 non-null  Int64              
 4   de_education_id           178921 non-null  object             
 5   de_Education_Type         178921 non-null  object             
 6   de_clientdemographicflag  600939 non-null  Int64              
 7   de_user_id                440197 non-null  object             
 8   de_device_id              440197 non-null  object             
 9   de_created_dt             440197 non-null  datetime64[us, UTC]
 10  de_gender                 440197 non-null  object             
 11  de_bi

In [25]:
sq = """drop table if exists worktable_data_analysis.customer_demographic_data;"""
client.query(sq)

# Convert 'onboardedDate' to a datetime format
res['de_onboardingDate'] = pd.to_datetime(res['de_onboardingDate'], errors='coerce')

# Define the dataset and table name
dataset_id = 'worktable_data_analysis'
table_id = 'customer_demographic_data'

# Define the schema based on the DataFrame structure
schema = [
    bigquery.SchemaField("cust_id", "STRING"),  
    bigquery.SchemaField("de_onboardingDate", "TIMESTAMP"), # Update to TIMESTAMP
    bigquery.SchemaField("de_kyc_status", "STRING"),
    bigquery.SchemaField("de_educationdataflag", "INT64"),
    bigquery.SchemaField("de_education_id", "STRING"),
    bigquery.SchemaField("de_Education_Type", "STRING"),
    bigquery.SchemaField("de_clientdemographicflag", "INT64"),
    bigquery.SchemaField("de_user_id", "STRING"),    
    bigquery.SchemaField("de_device_id", "STRING"),    
    bigquery.SchemaField("de_created_dt", "TIMESTAMP"),
    bigquery.SchemaField("de_gender", "STRING"),
    bigquery.SchemaField("de_birthplace", "STRING"),
    bigquery.SchemaField("de_mobile_no", "STRING"),
    bigquery.SchemaField("de_product", "STRING"),    
    bigquery.SchemaField("de_type", "STRING"),    
    bigquery.SchemaField("de_telcoProvider", "STRING"),    
    bigquery.SchemaField("de_cdd_flag", "INT64"),
    bigquery.SchemaField("de_dateOfBirth", "STRING"), 
    bigquery.SchemaField("de_signUpAccNo", "STRING"), 
    bigquery.SchemaField("de_kycStatusfromCDD", "STRING"),             
    bigquery.SchemaField("de_addressline1", "STRING"),
    bigquery.SchemaField("de_city", "STRING"),                   
    bigquery.SchemaField("de_province", "STRING"),  
    bigquery.SchemaField("de_barangay", "STRING"),             
    bigquery.SchemaField("de_provinceCode", "STRING"), 
    bigquery.SchemaField("de_postalcode", "STRING"),            
    bigquery.SchemaField("de_mailingCity", "STRING"),
    bigquery.SchemaField("de_mailingPostalCode", "STRING"), 
    bigquery.SchemaField("de_mailingProvince", "STRING"), 
    bigquery.SchemaField("de_natureofwork", "STRING"), 
    bigquery.SchemaField("de_employmentstatus", "STRING"), 
    bigquery.SchemaField("de_mobileOs", "STRING"), 
    bigquery.SchemaField("de_docType", "STRING"), 
    bigquery.SchemaField("de_Minutestakentoonboard", "INT64"),
    bigquery.SchemaField("de_companyName", "STRING"),   
    bigquery.SchemaField("de_industry", "STRING"),       
    bigquery.SchemaField("de_device_dtl", "STRING"),     
    bigquery.SchemaField("de_Region_name", "STRING"),     
    bigquery.SchemaField("de_locationdfflag", "INT64"),     
    bigquery.SchemaField("de_Onboarding_latitude", "STRING"),           
    bigquery.SchemaField("de_Onboarding_longitude", "STRING"),                    
    bigquery.SchemaField("de_loanjourney_latitude", "STRING"),        
    bigquery.SchemaField("de_loanjouney_longitude", "STRING"),     
    bigquery.SchemaField("de_ApigeeLogs_latitude", "STRING"),   
    bigquery.SchemaField("de_ApigeeLogs_longitude", "STRING"),   
    bigquery.SchemaField("de_loandfFlag", "INT64"),          
    bigquery.SchemaField("de_maritalStatus", "STRING"),          
    bigquery.SchemaField("de_dependentsCount", "STRING"),                 
    bigquery.SchemaField("de_monthlyIncome", "STRING"), 
    bigquery.SchemaField("de_startApplyDateTime", "TIMESTAMP"),         
    bigquery.SchemaField("de_network_group", "STRING"), 
    bigquery.SchemaField("de_Source", "STRING"), 
    bigquery.SchemaField("de_media_source", "STRING"),         
    bigquery.SchemaField("de_partner", "STRING"),          
    bigquery.SchemaField("de_campaign", "STRING"),    
    bigquery.SchemaField("de_rnk", "INT64"),            
    bigquery.SchemaField("de_Age", "FLOAT64"),                     
    bigquery.SchemaField("de_SourceGroups", "STRING"),                     
    bigquery.SchemaField("de_earliest_date", "TIMESTAMP"),                            
    bigquery.SchemaField("de_months_difference", "FLOAT64"), 
]

# Create the dataset reference
dataset_ref = client.dataset(dataset_id)

# Define the table reference
table_ref = dataset_ref.table(table_id)

# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig(schema=schema)

# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(res, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

print(f"Table {table_id} created in dataset {dataset_id}.")


Table customer_demographic_data created in dataset worktable_data_analysis.


# CredoLab Data

## Function to create the 12th day date and 150th day date

In [26]:
def calculate_dates(onboarding_date):
    # Convert the input date to datetime format
    onboarding_date = pd.to_datetime(onboarding_date)
    
    # Calculate the 120th and 150th days from the onboarding date
    day_120 = onboarding_date + pd.Timedelta(days=120)
    day_150 = onboarding_date + pd.Timedelta(days=150)
    
    return day_120, day_150

In [27]:
dfapp = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Customer_360_Data_Prep\Data\SupportingData\PH_TonikBank_applications_20240807.csv")

## cusmtb - Customer Basic information from credo lab

In [28]:
sq = """     
with cusmtb as
(select distinct cust_id, date(created_dt) onboardingDate 
, date_add(date(created_dt), INTERVAL 120 day) day_120
, date_add(date(created_dt), INTERVAL 150 day) day_150
FROM `dl_customers_db_raw.tdbk_customer_mtb` 
WHERE 1=1 and date(created_dt) >= '2023-01-01'
and date(created_dt) <= '2024-03-31'
and cust_id is not null
),
base as 
(select 
b.cust_id ,
        b.onboardingDate,
        b.day_120,
        b.day_150,
        tlcd.kycStatus      ,
        t3.creditScoreUpdated   ,
        t3.fraudScore   ,	
        t3.fraudScoreUpdated    ,
        t3.calculateddate   ,
        t4.run_date ,
        ca.package_name ,
        ca.first_install_time    ,
        t4.GeneralInfo.brand     ,
        t4.Hardware.device__brand   ,
        t4.Hardware.device__manufacturer   ,
        t4.Hardware.device__model,
        t4.GeneralData.telephony_info__network_operator_name,
        t4.GeneralData.telephony_info__network_operator,
        t4.GeneralData.sim_operator_name,
    from cusmtb b
    inner join prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_customer_details tlcd on tlcd.custId = b.cust_id  
    inner JOIN `prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3 ON cast(tlcd.credolabRefNumber as string) = cast(t3.refno as string) and date(t3.createdOn) <= b.day_120
    left JOIN `prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4 ON t3.refno = t4.deviceId and date(run_date) <= b.day_120
    INNER JOIN
    (select deviceId, af.package_name as package_name, af.first_install_time as first_install_time from `prj-prod-dataplatform.credolab_raw.android_credolab_Application`  ,
    unnest(Application) as af) ca ON ca.deviceId = t3.refno and date(run_date) <= b.day_120
),
base2 as
(select distinct cust_id, onboardingDate, day_120, day_150, kycStatus, creditScoreUpdated, fraudScore , fraudScoreUpdated, run_date 
, brand, device__brand, device__manufacturer, device__model
, telephony_info__network_operator_name, telephony_info__network_operator, sim_operator_name 
from base
)
select * from base2;
"""

cusmtb = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of cusmtb is :\t{cusmtb.shape}")
cusmtb.to_csv(r"cusmtb.csv", index = False)

Job ID 36974ad5-1774-46cb-95c6-be014dc55b5a successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of cusmtb is :	(304260, 16)


## cusfirsttimeinstall  - Min first install time

In [29]:
sq = """with cusmtb as
(select distinct cust_id, date(created_dt) onboardingDate 
, date_add(date(created_dt), INTERVAL 120 day) day_120
, date_add(date(created_dt), INTERVAL 150 day) day_150
FROM `dl_customers_db_raw.tdbk_customer_mtb` 
WHERE 1=1 and date(created_dt) >= '2023-01-01' 
and date(created_dt) <= '2024-03-31'
and cust_id is not null
),
base as 
(select 
b.cust_id ,
        b.onboardingDate,
        b.day_120,
        b.day_150,
        tlcd.kycStatus      ,
        t3.creditScoreUpdated   ,
        t3.fraudScore   ,	
        t3.fraudScoreUpdated    ,
        t3.calculateddate   ,
        t4.run_date ,
        ca.package_name ,
        ca.first_install_time    ,
        t4.GeneralInfo.brand     ,
        t4.Hardware.device__brand   ,
        t4.Hardware.device__manufacturer   ,
        t4.Hardware.device__model,
        t4.GeneralData.telephony_info__network_operator_name,
        t4.GeneralData.telephony_info__network_operator,
        t4.GeneralData.sim_operator_name,
    from cusmtb b
    inner join prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_customer_details tlcd on tlcd.custId = b.cust_id  
    inner JOIN `prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3 ON cast(tlcd.credolabRefNumber as string) = cast(t3.refno as string) and date(t3.createdOn) <= b.day_120
    left JOIN `prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4 ON t3.refno = t4.deviceId and date(run_date) <= b.day_120
    INNER JOIN
    (select deviceId, af.package_name as package_name, af.first_install_time as first_install_time from `prj-prod-dataplatform.credolab_raw.android_credolab_Application`  ,
    unnest(Application) as af) ca ON ca.deviceId = t3.refno and date(run_date) <= b.day_120
),
base2 as
(select distinct cust_id, onboardingDate, day_120, day_150, kycStatus, creditScoreUpdated, fraudScore , fraudScoreUpdated, run_date 
, brand, device__brand, device__manufacturer, device__model
, telephony_info__network_operator_name, telephony_info__network_operator, sim_operator_name 
from base
)
-- select * from base2 where cust_id = '2242663';
-- select cust_id, count(cust_id) from base2 group by 1 having count(cust_id) > 1;
----There is no duplicate in base2
, base3 as 
(select cust_id, min(first_install_time) first_install_time from base group by 1)
select * from base3;"""
cusfirsttimeinstall = client.query(sq).to_dataframe(progress_bar_type='tqdm')


Job ID b184d347-bf59-4ed7-ba58-0095b612e2ac successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [30]:
print(f"The shape of cusfirsttimeinstall is :\t{cusfirsttimeinstall.shape}")
cusfirsttimeinstall.to_csv(r"cusfirsttimeinstall.csv", index = False)

The shape of cusfirsttimeinstall is :	(304260, 2)


## results - For data related to package details for customers

I have to run this in bigquery notebook and download the csv file there and then download it back to my laptop for further processing

In [31]:
# sq = """with cusmtb as
# (select distinct cust_id, date(created_dt) onboardingDate 
# , date_add(date(created_dt), INTERVAL 120 day) day_120
# , date_add(date(created_dt), INTERVAL 150 day) day_150
# FROM `dl_customers_db_raw.tdbk_customer_mtb` 
# WHERE 1=1 and date(created_dt) >= '2023-01-01' and date(created_dt) <= '2024-03-31' and cust_id is not null
# ),
# base as 
# (select 
# b.cust_id ,
#         b.onboardingDate,
#         b.day_120,
#         b.day_150,
#         tlcd.kycStatus      ,
#         t3.creditScoreUpdated   ,
#         t3.fraudScore   ,	
#         t3.fraudScoreUpdated    ,
#         t3.calculateddate   ,
#         t4.run_date ,
#         ca.package_name ,
#         ca.first_install_time    ,
#         t4.GeneralInfo.brand     ,
#         t4.Hardware.device__brand   ,
#         t4.Hardware.device__manufacturer   ,
#         t4.Hardware.device__model,
#         t4.GeneralData.telephony_info__network_operator_name,
#         t4.GeneralData.telephony_info__network_operator,
#         t4.GeneralData.sim_operator_name,
#     from cusmtb b
#     inner join prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_customer_details tlcd on tlcd.custId = b.cust_id  
#     inner JOIN `prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3 ON cast(tlcd.credolabRefNumber as string) = cast(t3.refno as string) and date(t3.createdOn) <= b.day_120
#     left JOIN `prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4 ON t3.refno = t4.deviceId and date(run_date) <= b.day_120
#     INNER JOIN
#     (select deviceId, af.package_name as package_name, af.first_install_time as first_install_time from `prj-prod-dataplatform.credolab_raw.android_credolab_Application`  ,
#     unnest(Application) as af) ca ON ca.deviceId = t3.refno and date(run_date) <= b.day_120
# ),
# base2 as
# (select distinct cust_id, onboardingDate, day_120, day_150, kycStatus, creditScoreUpdated, fraudScore , fraudScoreUpdated, run_date 
# , brand, device__brand, device__manufacturer, device__model
# , telephony_info__network_operator_name, telephony_info__network_operator, sim_operator_name 
# from base
# )
# , base3 as 
# (select cust_id, min(first_install_time) first_install_time from base group by 1),
# base4 as 
# (select distinct cust_id, package_name from base )
# select * from base4;
# """
# cuspackage = client.query(sq).to_dataframe(progress_bar_type='tqdm')
# print(f"The shape of cuspackage is :\t{cuspackage.shape}")
# cuspackage.to_csv(r"cuspackage.csv", index = False)

## Read the result csv file

In [32]:
results = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Customer_360_Data_Prep\Data\SupportingData\results.csv")

## Read the package information csv file provided by credolab for package categorization

In [33]:
dfapp = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Customer_360_Data_Prep\Data\SupportingData\PH_TonikBank_applications_20240807.csv")

In [34]:
dfapp.head()

,Package Name,# of datasets,# of datasets where app is considered as user installed,Category,Rating,Rating Description,Is Paid
0,com.tonik.mobile,685556,685553,finance,rated for 3+,undefined,free
1,android,684929,0,NaN,NaN,NaN,NaN
2,com.android.providers.settings,684922,0,NaN,NaN,NaN,NaN
3,com.android.externalstorage,684920,0,NaN,NaN,NaN,NaN
4,com.android.settings,684920,0,NaN,NaN,NaN,NaN


In [35]:
dfapp['Category'].unique()

array(['finance', nan, 'tools', 'business', 'communication',
       'video players & editors', 'travel & local', 'social',
       'productivity', 'photography', 'auto & vehicles', 'shopping',
       'music & audio', 'entertainment', 'action', 'news & magazines',
       'food & drink', 'lifestyle', 'personalization', 'art & design',
       'maps & navigation', 'adventure', 'weather', 'health & fitness',
       'casual', 'education', 'sports', 'strategy', 'casino', 'puzzle',
       'books & reference', 'word', 'arcade', 'educational', 'medical',
       'card', 'role playing', 'dating', 'comics', 'board', 'music',
       'simulation', 'racing', 'trivia', 'parenting', 'events', 'beauty',
       'house & home', 'libraries & demo', 'undefined'], dtype=object)

In [36]:
import re

def categorize_package(package_name):
    if isinstance(package_name, str):
        pattern = r'linkedin|jobstreet|glint'
        if re.search(pattern, package_name, re.IGNORECASE):
            return 'Job'
        pattern1 = r'office|gdrive|word|powerpoint|excel|'
        if re.search(pattern, package_name, re.IGNORECASE):
            return 'Professional'
    return 'Other'

dfapp['CustomCategory'] = dfapp['Package Name'].apply(categorize_package)
dfapp.head()

,Package Name,# of datasets,# of datasets where app is considered as user installed,Category,Rating,Rating Description,Is Paid,CustomCategory
0,com.tonik.mobile,685556,685553,finance,rated for 3+,undefined,free,Other
1,android,684929,0,NaN,NaN,NaN,NaN,Other
2,com.android.providers.settings,684922,0,NaN,NaN,NaN,NaN,Other
3,com.android.externalstorage,684920,0,NaN,NaN,NaN,NaN,Other
4,com.android.settings,684920,0,NaN,NaN,NaN,NaN,Other


In [37]:
dfapp['CombinedCategory'] = np.where((dfapp['CustomCategory'].notna()) & (dfapp['CustomCategory'] != 'Other'),
                                     dfapp['CustomCategory'],
                                     dfapp['Category'])

In [38]:
dfapp.drop(columns=['Category', 'CustomCategory'], inplace=True)
dfapp.rename(columns={'CombinedCategory':'Category'}, inplace = True)
dfapp.head()

,Package Name,# of datasets,# of datasets where app is considered as user installed,Rating,Rating Description,Is Paid,Category
0,com.tonik.mobile,685556,685553,rated for 3+,undefined,free,finance
1,android,684929,0,NaN,NaN,NaN,NaN
2,com.android.providers.settings,684922,0,NaN,NaN,NaN,NaN
3,com.android.externalstorage,684920,0,NaN,NaN,NaN,NaN
4,com.android.settings,684920,0,NaN,NaN,NaN,NaN


In [39]:
# For df_risk_table_2
results['package'] = results['package_name'].str.replace(r'[ ._]', '', regex=True)

# For dfapp
dfapp['package'] = dfapp['Package Name'].str.replace(r'[ ._]', '', regex=True)


In [40]:
dfapp.columns

Index(['Package Name', '# of datasets',
       '# of datasets where app is considered as user installed', 'Rating',
       'Rating Description', 'Is Paid', 'Category', 'package'],
      dtype='object')

In [41]:
dfa = dfapp[['package', 'Category', 'Rating','Rating Description', 'Is Paid']].copy()

In [42]:
Credodata = results.merge(dfa, on='package', how = 'outer')

In [43]:
Credodata.head()

,cust_id,package_name,package,Category,Rating,Rating Description,Is Paid
0,NaN,NaN,ACENAPK,NaN,NaN,NaN,NaN
1,2201933.0,AM.callernameannouncer.callernamespeaker,AMcallernameannouncercallernamespeaker,NaN,NaN,NaN,NaN
2,NaN,NaN,AO3ArchiveofOurOwnhwawi,NaN,NaN,NaN,NaN
3,NaN,NaN,ARRulerBirdmanStudionet,NaN,NaN,NaN,NaN
4,1907498.0,AcrylicNails.huawei,AcrylicNailshuawei,NaN,NaN,NaN,NaN


## Check the category wise unique count of customer id

In [44]:
Credodata.groupby('Category')['cust_id'].nunique().sort_values(ascending = False)

Category
tools                      301891
communication              282929
finance                    268781
productivity               263906
video players & editors    232980
social                     196271
music & audio              181230
business                   177089
shopping                   174701
photography                166829
entertainment              166793
travel & local             157398
auto & vehicles             92462
lifestyle                   83014
casual                      75665
action                      74058
maps & navigation           72704
personalization             68502
health & fitness            58894
food & drink                58109
news & magazines            53699
books & reference           46150
education                   45502
puzzle                      44238
Job                         38633
arcade                      34101
simulation                  28339
strategy                    24806
adventure                   24251
casin

## Check the rating wise unique count of customer id

In [45]:
Credodata.groupby(['Rating'])['cust_id'].nunique().sort_values(ascending = False)

Rating
rated for 3+     304238
rated for 12+    299217
rated for 18+     92870
rated for 7+      57719
rated for 16+     39868
undefined            33
unrated               3
Name: cust_id, dtype: int64

In [46]:
Credodata.columns

Index(['cust_id', 'package_name', 'package', 'Category', 'Rating',
       'Rating Description', 'Is Paid'],
      dtype='object')

In [47]:
Credodata['Category'] = Credodata['Category'].fillna('Others')

In [48]:
Credodata[Credodata['cust_id'].isnull() == True]

,cust_id,package_name,package,Category,Rating,Rating Description,Is Paid
0,NaN,NaN,ACENAPK,Others,NaN,NaN,NaN
2,NaN,NaN,AO3ArchiveofOurOwnhwawi,Others,NaN,NaN,NaN
3,NaN,NaN,ARRulerBirdmanStudionet,Others,NaN,NaN,NaN
9,NaN,NaN,AutoLoanCalculatorPRODPsoftwareorg,Others,NaN,NaN,NaN
12,NaN,NaN,BIMobileWombat,Others,NaN,NaN,NaN
...,...,...,...,...,...,...,...
31758063,NaN,NaN,zygonstreamcomapp,music & audio,rated for 3+,undefined,free
31758064,NaN,NaN,zyxdolindachat,Others,NaN,NaN,NaN
31758065,NaN,NaN,zyxxyballbox,Others,NaN,NaN,NaN
31758067,NaN,NaN,zzmdgclelectroniccigarette,tools,rated for 3+,undefined,free


## Drop all rows where cust_id is null

In [49]:
Credodata = Credodata.dropna(subset=['cust_id'])

In [50]:
Credodata.shape

(31491899, 7)

In [51]:
# Find all rows with duplicates (including the first occurrence)
all_duplicate_rows = Credodata[Credodata.duplicated(keep=False)]

# Display all duplicate rows
all_duplicate_rows.head()

,cust_id,package_name,package,Category,Rating,Rating Description,Is Paid
446480,1854806.0,business.ideas,businessideas,business,rated for 3+,undefined,free
446481,1854806.0,business.ideas,businessideas,business,rated for 3+,undefined,free
446482,1959976.0,business.ideas,businessideas,business,rated for 3+,undefined,free
446483,1959976.0,business.ideas,businessideas,business,rated for 3+,undefined,free
463751,2132428.0,cn.oneplus.oemtcma,cnoneplusoemtcma,Others,NaN,NaN,NaN


## Create each row for each unique package found for customer id with the number of unique different packages found in each category

### dfapp1

In [52]:
dfapp1 = Credodata.pivot_table(index = 'cust_id', columns='Category', values='package', aggfunc='nunique').reset_index()
dfapp1.head()

Category,cust_id,Job,Others,action,adventure,arcade,art & design,auto & vehicles,beauty,board,books & reference,business,card,casino,casual,comics,communication,dating,education,educational,entertainment,events,finance,food & drink,health & fitness,house & home,libraries & demo,lifestyle,maps & navigation,medical,music,music & audio,news & magazines,parenting,personalization,photography,productivity,puzzle,racing,role playing,shopping,simulation,social,sports,strategy,tools,travel & local,trivia,undefined,video players & editors,weather,word
0,1845909.0,NaN,73.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,4.0,NaN,1.0,NaN,1.0,NaN,7.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,3.0,NaN,NaN,1.0,1.0,3.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,10.0,2.0,NaN,NaN,2.0,NaN,NaN
1,1845911.0,NaN,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,6.0,NaN,NaN,NaN,1.0,NaN,NaN
2,1845913.0,NaN,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,2.0,NaN,NaN
3,1845915.0,NaN,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,6.0,1.0,NaN,NaN,4.0,NaN,NaN
4,1845916.0,NaN,65.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN


In [53]:
dfapp1 = dfapp1.fillna(0.0)

In [54]:
# Convert to float first
dfapp1['cust_id'] = dfapp1['cust_id'].astype(float)

# Convert to integer (rounding if necessary) and then to string
dfapp1['cust_id'] = dfapp1['cust_id'].round(0).astype(int).astype(str)

In [55]:
dfapp1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304331 entries, 0 to 304330
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   cust_id                  304331 non-null  object 
 1   Job                      304331 non-null  float64
 2   Others                   304331 non-null  float64
 3   action                   304331 non-null  float64
 4   adventure                304331 non-null  float64
 5   arcade                   304331 non-null  float64
 6   art & design             304331 non-null  float64
 7   auto & vehicles          304331 non-null  float64
 8   beauty                   304331 non-null  float64
 9   board                    304331 non-null  float64
 10  books & reference        304331 non-null  float64
 11  business                 304331 non-null  float64
 12  card                     304331 non-null  float64
 13  casino                   304331 non-null  float64
 14  casu

In [56]:
dfapp1 = dfapp1[['cust_id', 'Others', 'action', 'adventure', 'arcade', 'art & design',
       'auto & vehicles', 'beauty', 'board', 'books & reference', 'business',
       'card', 'casino', 'casual', 'comics', 'communication', 'dating',
       'education', 'educational', 'entertainment', 'finance', 'food & drink',
       'health & fitness', 'lifestyle', 'maps & navigation', 'medical',
       'music', 'music & audio', 'news & magazines', 'personalization',
       'photography', 'productivity', 'puzzle', 'racing', 'role playing',
       'shopping', 'simulation', 'social', 'sports', 'strategy', 'tools',
       'travel & local', 'trivia', 'video players & editors', 'weather',
       'word']].copy()

## dfrating

In [57]:
dfrating = Credodata.pivot_table(index = 'cust_id', columns='Rating', values='package', aggfunc='nunique').reset_index()
dfrating.head()

Rating,cust_id,rated for 12+,rated for 16+,rated for 18+,rated for 3+,rated for 7+,undefined,unrated
0,1845909.0,9.0,NaN,NaN,35.0,NaN,NaN,NaN
1,1845911.0,4.0,NaN,NaN,18.0,NaN,NaN,NaN
2,1845913.0,1.0,NaN,1.0,19.0,NaN,NaN,NaN
3,1845915.0,7.0,NaN,NaN,12.0,NaN,NaN,NaN
4,1845916.0,6.0,1.0,NaN,7.0,NaN,NaN,NaN


In [58]:
dfrating1 = dfrating[['cust_id', 'rated for 12+', 'rated for 16+', 'rated for 18+']].copy()

In [59]:
# Convert to float first
dfrating1['cust_id'] = dfrating1['cust_id'].astype(float)

# Convert to integer (rounding if necessary) and then to string
dfrating1['cust_id'] = dfrating1['cust_id'].round(0).astype(int).astype(str)

## dfpaid

In [60]:
dfpaid = Credodata.pivot_table(index = 'cust_id', columns='Is Paid', values='package', aggfunc='nunique').reset_index()
dfpaid = dfpaid.fillna(0)

In [61]:
# Convert to float first
dfpaid['cust_id'] = dfpaid['cust_id'].astype(float)

# Convert to integer (rounding if necessary) and then to string
dfpaid['cust_id'] = dfpaid['cust_id'].round(0).astype(int).astype(str)

In [62]:
print(f"The shape of cusmtb is :\t {cusmtb.shape}")
print(f"The shape of cusfirsttimeinstall is:\t {cusfirsttimeinstall.shape}")
print(f"The shape of dfapp1 is:\t{dfapp1.shape}")
print(f"The shape of dfrating1 is:\t {dfrating1.shape}")
print(f"The shape of dfpaid is:\t{dfpaid.shape}")

The shape of cusmtb is :	 (304260, 16)
The shape of cusfirsttimeinstall is:	 (304260, 2)
The shape of dfapp1 is:	(304331, 46)
The shape of dfrating1 is:	 (304289, 4)
The shape of dfpaid is:	(304289, 4)


In [63]:
cusmtb.head()

,cust_id,onboardingDate,day_120,day_150,kycStatus,creditScoreUpdated,fraudScore,fraudScoreUpdated,run_date,brand,device__brand,device__manufacturer,device__model,telephony_info__network_operator_name,telephony_info__network_operator,sim_operator_name
0,1856869,2023-01-10,2023-05-10,2023-06-09,SKYC,Y,944.0,Y,2023-01-10,Infinix,Infinix,None,Infinix X682C,du,42403,du
1,2061547,2023-05-25,2023-09-22,2023-10-22,SKYC,Y,944.0,Y,2023-05-25,samsung,samsung,None,SM-G981V,etisalat,42402,Etisalat
2,2321497,2023-12-02,2024-03-31,2024-04-30,SKYC,None,944.0,None,2023-12-02,OPPO,OPPO,None,CPH2481,Sun,51503,SUN
3,2089820,2023-06-12,2023-10-10,2023-11-09,SKYC,Y,944.0,Y,2023-06-12,lge,lge,None,LM-G900N,NTT DOCOMO,44010,Smart Communications
4,2305419,2023-11-15,2024-03-14,2024-04-13,SKYC,None,824.0,None,2023-11-15,samsung,samsung,None,SM-A226B,etisalat,42402,Etisalat


In [64]:
finaldf  = cusmtb.merge(cusfirsttimeinstall, on='cust_id', how = 'left')
finaldf = finaldf.merge(dfapp1, on='cust_id', how = 'left')
finaldf = finaldf.merge(dfrating1, on='cust_id', how = 'left')
finaldf = finaldf.merge(dfpaid , on='cust_id', how ='left')

In [65]:
finaldf.head()

,cust_id,onboardingDate,day_120,day_150,kycStatus,creditScoreUpdated,fraudScore,fraudScoreUpdated,run_date,brand,device__brand,device__manufacturer,device__model,telephony_info__network_operator_name,telephony_info__network_operator,sim_operator_name,first_install_time,Others,action,adventure,arcade,art & design,auto & vehicles,beauty,board,books & reference,business,card,casino,casual,comics,communication,dating,education,educational,entertainment,finance,food & drink,health & fitness,lifestyle,maps & navigation,medical,music,music & audio,news & magazines,personalization,photography,productivity,puzzle,racing,role playing,shopping,simulation,social,sports,strategy,tools,travel & local,trivia,video players & editors,weather,word,rated for 12+,rated for 16+,rated for 18+,free,pay,undefined
0,1856869,2023-01-10,2023-05-10,2023-06-09,SKYC,Y,944.0,Y,2023-01-10,Infinix,Infinix,None,Infinix X682C,du,42403,du,2009-01-01 08:00:00+00:00,68.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,5.0,NaN,NaN,16.0,0.0,0.0
1,2061547,2023-05-25,2023-09-22,2023-10-22,SKYC,Y,944.0,Y,2023-05-25,samsung,samsung,None,SM-G981V,etisalat,42402,Etisalat,1970-01-01 08:00:00+00:00,85.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,3.0,NaN,1.0,25.0,0.0,0.0
2,2321497,2023-12-02,2024-03-31,2024-04-30,SKYC,None,944.0,None,2023-12-02,OPPO,OPPO,None,CPH2481,Sun,51503,SUN,1970-01-01 08:00:00+00:00,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,7.0,NaN,NaN,29.0,0.0,0.0
3,2089820,2023-06-12,2023-10-10,2023-11-09,SKYC,Y,944.0,Y,2023-06-12,lge,lge,None,LM-G900N,NTT DOCOMO,44010,Smart Communications,1970-01-01 08:00:00+00:00,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,6.0,NaN,1.0,17.0,0.0,0.0
4,2305419,2023-11-15,2024-03-14,2024-04-13,SKYC,None,824.0,None,2023-11-15,samsung,samsung,None,SM-A226B,etisalat,42402,Etisalat,1970-01-01 08:00:00+00:00,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,23.0,0.0,0.0


## Select only SKYC customer

In [66]:
finaldf['kycStatus'].value_counts()

kycStatus
SKYC    270105
BKYC     34154
FKYC         1
Name: count, dtype: int64

In [67]:
finaldf = finaldf[finaldf['kycStatus'] != 'BKYC']
finaldf['kycStatus'].value_counts()

kycStatus
SKYC    270105
FKYC         1
Name: count, dtype: int64

In [68]:
finaldf.to_csv("Credolabdata20240913.csv", index = False)

## Rename column name

In [69]:
# Read the Excel file (replace 'your_file.xlsx' with the actual file name)
excel_data = pd.read_csv(r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Customer_360_Data_Prep\Data\SupportingData\Credolabdatacolumnrename.csv')
# excel_data.columns
# Create a dictionary to map current column names to updated column names
rename_dict = dict(zip(excel_data['Columnname'], excel_data['Updated_Name']))

rename_dict

{'cust_id': 'cust_id',
 'onboardingDate': 'onboardingDate',
 'day_120': 'cl_day_120',
 'day_150': 'cl_day_150',
 'kycStatus': 'cl_kycStatus',
 'creditScoreUpdated': 'cl_creditScoreUpdated',
 'fraudScore': 'cl_fraudScore',
 'fraudScoreUpdated': 'cl_fraudScoreUpdated',
 'run_date': 'cl_run_date',
 'brand': 'cl_brand',
 'device__brand': 'cl_device__brand',
 'device__manufacturer': 'cl_device__manufacturer',
 'device__model': 'cl_device__model',
 'telephony_info__network_operator_name': 'cl_telephony_info__network_operator_name',
 'telephony_info__network_operator': 'cl_telephony_info__network_operator',
 'sim_operator_name': 'cl_sim_operator_name',
 'first_install_time': 'cl_first_install_time',
 'Others': 'cl_pkg_Others',
 'action': 'cl_pkg_action',
 'adventure': 'cl_pkg_adventure',
 'arcade': 'cl_pkg_arcade',
 'art & design': 'cl_pkg_art _ design',
 'auto & vehicles': 'cl_pkg_auto _ vehicles',
 'beauty': 'cl_pkg_beauty',
 'board': 'cl_pkg_board',
 'books & reference': 'cl_pkg_books _ re

## Rename the dataframe column

In [70]:
# Rename columns in the DataFrame 'res'
finaldf.rename(columns=rename_dict, inplace=True)
finaldf.columns

Index(['cust_id', 'onboardingDate', 'cl_day_120', 'cl_day_150', 'cl_kycStatus',
       'cl_creditScoreUpdated', 'cl_fraudScore', 'cl_fraudScoreUpdated',
       'cl_run_date', 'cl_brand', 'cl_device__brand',
       'cl_device__manufacturer', 'cl_device__model',
       'cl_telephony_info__network_operator_name',
       'cl_telephony_info__network_operator', 'cl_sim_operator_name',
       'cl_first_install_time', 'cl_pkg_Others', 'cl_pkg_action',
       'cl_pkg_adventure', 'cl_pkg_arcade', 'cl_pkg_art _ design',
       'cl_pkg_auto _ vehicles', 'cl_pkg_beauty', 'cl_pkg_board',
       'cl_pkg_books _ reference', 'cl_pkg_business', 'cl_pkg_card',
       'cl_pkg_casino', 'cl_pkg_casual', 'cl_pkg_comics',
       'cl_pkg_communication', 'cl_pkg_dating', 'cl_pkg_education',
       'cl_pkg_educational', 'cl_pkg_entertainment', 'cl_pkg_finance',
       'cl_pkg_food _ drink', 'cl_pkg_health _ fitness', 'cl_pkg_lifestyle',
       'cl_pkg_maps _ navigation', 'cl_pkg_medical', 'cl_pkg_music',
       

In [71]:
finaldf.to_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\Customer_360_Data_Prep\Data\FinalData\Credolabdatawithrenamedcolumns20240913.csv", index = False)

In [72]:
finaldf[finaldf['cust_id'].isnull()]

,cust_id,onboardingDate,cl_day_120,cl_day_150,cl_kycStatus,cl_creditScoreUpdated,cl_fraudScore,cl_fraudScoreUpdated,cl_run_date,cl_brand,cl_device__brand,cl_device__manufacturer,cl_device__model,cl_telephony_info__network_operator_name,cl_telephony_info__network_operator,cl_sim_operator_name,cl_first_install_time,cl_pkg_Others,cl_pkg_action,cl_pkg_adventure,cl_pkg_arcade,cl_pkg_art _ design,cl_pkg_auto _ vehicles,cl_pkg_beauty,cl_pkg_board,cl_pkg_books _ reference,cl_pkg_business,cl_pkg_card,cl_pkg_casino,cl_pkg_casual,cl_pkg_comics,cl_pkg_communication,cl_pkg_dating,cl_pkg_education,cl_pkg_educational,cl_pkg_entertainment,cl_pkg_finance,cl_pkg_food _ drink,cl_pkg_health _ fitness,cl_pkg_lifestyle,cl_pkg_maps _ navigation,cl_pkg_medical,cl_pkg_music,cl_pkg_music _ audio,cl_pkg_news _ magazines,cl_pkg_personalization,cl_pkg_photography,cl_pkg_productivity,cl_pkg_puzzle,cl_pkg_racing,cl_pkg_roleplaying,cl_pkg_shopping,cl_pkg_simulation,cl_pkg_social,cl_pkg_sports,cl_pkg_strategy,cl_pkg_tools,cl_pkg_travel _ local,cl_pkg_trivia,cl_pkg_videoplayers _ editors,cl_pkg_weather,cl_pkg_word,cl_pkg_rated for 12_plus,cl_pkg_rated for 16_plus,cl_pkg_rated for 18_plus,cl_pkg_free,cl_pkg_pay,cl_pkg_undefined


In [73]:
finaldf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 270106 entries, 0 to 304259
Data columns (total 68 columns):
 #   Column                                    Non-Null Count   Dtype              
---  ------                                    --------------   -----              
 0   cust_id                                   270106 non-null  object             
 1   onboardingDate                            270106 non-null  dbdate             
 2   cl_day_120                                270106 non-null  dbdate             
 3   cl_day_150                                270106 non-null  dbdate             
 4   cl_kycStatus                              270106 non-null  object             
 5   cl_creditScoreUpdated                     102047 non-null  object             
 6   cl_fraudScore                             270088 non-null  object             
 7   cl_fraudScoreUpdated                      102047 non-null  object             
 8   cl_run_date                               270106 

In [74]:
finaldf.rename(columns = {'cl_pkg_rated for 12_plus':'cl_pkg_rated_for_12_plus', 'cl_pkg_rated for 16_plus':'cl_pkg_rated_for_16_plus', 'cl_pkg_rated for 18_plus':'cl_pkg_rated_for_18_plus'}, inplace = True)

In [75]:
# Drop the existing table if it exists
sq = """DROP TABLE IF EXISTS prj-prod-dataplatform.worktable_data_analysis.customer_credolab_data;"""
client.query(sq)

# Convert the 'onboardingDate' column to a datetime format
finaldf['onboardingDate'] = pd.to_datetime(finaldf['onboardingDate'], errors='coerce')

# Define the dataset and table names
dataset_id = 'worktable_data_analysis'
table_id = 'customer_credolab_data'

# Define the schema based on the DataFrame structure
schema = [
    bigquery.SchemaField("cust_id", "STRING"),  
    bigquery.SchemaField("onboardingDate", "TIMESTAMP"), # Assuming the date fields to be converted to TIMESTAMP
    bigquery.SchemaField("cl_day_120", "TIMESTAMP"),
    bigquery.SchemaField("cl_day_150", "TIMESTAMP"),
    bigquery.SchemaField("cl_kycStatus", "STRING"),
    bigquery.SchemaField("cl_creditScoreUpdated", "STRING"),
    bigquery.SchemaField("cl_fraudScore", "STRING"),
    bigquery.SchemaField("cl_fraudScoreUpdated", "STRING"),
    bigquery.SchemaField("cl_run_date", "TIMESTAMP"),
    bigquery.SchemaField("cl_brand", "STRING"),
    bigquery.SchemaField("cl_device__brand", "STRING"),
    bigquery.SchemaField("cl_device__manufacturer", "STRING"),
    bigquery.SchemaField("cl_device__model", "STRING"),
    bigquery.SchemaField("cl_telephony_info__network_operator_name", "STRING"),
    bigquery.SchemaField("cl_telephony_info__network_operator", "STRING"),
    bigquery.SchemaField("cl_sim_operator_name", "STRING"),
    bigquery.SchemaField("cl_first_install_time", "TIMESTAMP"),
    bigquery.SchemaField("cl_pkg_Others", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_action", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_adventure","FLOAT64"),   
    bigquery.SchemaField("cl_pkg_arcade", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_art _ design", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_auto _ vehicles","FLOAT64"),
    bigquery.SchemaField("cl_pkg_beauty","FLOAT64"),
    bigquery.SchemaField("cl_pkg_board", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_books _ reference", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_business", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_card", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_casino", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_casual", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_comics", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_communication", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_dating", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_education", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_educational", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_entertainment", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_finance", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_food _ drink", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_health _ fitness", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_lifestyle", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_maps _ navigation", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_medical", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_music", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_music _ audio", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_news _ magazines", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_personalization", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_photography", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_productivity", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_puzzle", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_racing", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_roleplaying", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_shopping", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_simulation", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_social", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_sports", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_strategy", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_tools", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_travel _ local", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_trivia", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_videoplayers _ editors", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_weather", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_word", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_rated_for_12_plus", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_rated_for_12_plus", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_rated_for_16_plus", "FLOAT64"),  # Corrected field name
    bigquery.SchemaField("cl_pkg_rated_for_18_plus", "FLOAT64"),  # Corrected field name    
    bigquery.SchemaField("cl_pkg_free", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_pay", "FLOAT64"),
    bigquery.SchemaField("cl_pkg_undefined", "FLOAT64"),
]

# Create the dataset reference
dataset_ref = client.dataset(dataset_id)

# Define the table reference
table_ref = dataset_ref.table(table_id)

# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig(schema=schema, write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE)

# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(finaldf, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

print(f"Table {table_id} created in dataset {dataset_id}.")


Table customer_credolab_data created in dataset worktable_data_analysis.


: 